In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import datetime
import copy
import json
import time
import os

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing import get_dataframes_cached

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.sensitivity import calculate_sensitivity_and_plot
from main.seir.forecast import get_forecast, forecast_all_trials, trials_to_df, scale_up_testing_and_forecast
from main.seir.forecast import create_all_csvs, create_decile_csv, create_region_csv, write_csv
from main.seir.uncertainty import MCUncertainty
from utils.create_report import create_report
from utils.enums import Columns
from utils.loss import Loss_Calculator
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

## Params

In [ ]:
date_of_interest = '2020-07-31'
forecast_days = 54
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
folder = str(now)
ktrials = 10
model=SEIRHD

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    state, district, train_period=21, val_period=7, num_evals=1000,
    data_from_tracker=True, granular_data=False, filename=None, initialisation='intermediate', model=model, 
    smooth_jump=True,
    which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])
predictions_dict['m2'] = single_fitting_cycle(
    state, district, train_period=21, val_period=0, num_evals=1000,
    data_from_tracker=True, granular_data=False, filename=None, initialisation='intermediate', model=model, 
    smooth_jump=True,
    which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']
predictions_dict['data_last_date'] = predictions_dict['m2']['data_last_date']

## Loss DataFrames

### M1 Loss DataFrame

In [ ]:
predictions_dict['m1']['df_loss']

### M2 Loss DataFrame

In [ ]:
predictions_dict['m2']['df_loss']

## Plot Forecasts

In [ ]:
predictions_dict['m2']['forecasts'] = {}
predictions_dict['m2']['forecasts']['best'] = get_forecast(predictions_dict, train_fit='m2', days=forecast_days)
predictions_dict['m2']['plots']['forecast_best'] = plot_forecast(predictions_dict, (state, district), error_bars=True)

trials_processed = forecast_all_trials(predictions_dict, train_fit='m1', forecast_days=forecast_days)
predictions_dict['m1']['trials_processed'] = trials_processed
predictions_dict['m1']['all_trials'] = trials_to_df(trials_processed, column=Columns.confirmed)
trials_processed = forecast_all_trials(predictions_dict, train_fit='m2', forecast_days=forecast_days)
predictions_dict['m2']['trials_processed'] = trials_processed
predictions_dict['m2']['all_trials'] = trials_to_df(trials_processed, column=Columns.confirmed)
kforecasts = plot_top_k_trials(
    predictions_dict,
    train_fit='m2',
    trials_processed=predictions_dict['m2']['trials_processed'], 
    k=ktrials,
    which_compartments=[Columns.confirmed, Columns.active])
predictions_dict['m2']['plots']['forecast_confirmed_topk'] = kforecasts[Columns.confirmed]
predictions_dict['m2']['plots']['forecast_active_topk'] = kforecasts[Columns.active]


## Uncertainty

In [ ]:
region_dict = predictions_dict[('Maharashtra', 'Mumbai')]
uncertainty = MCUncertainty(region_dict, date_of_interest)

In [ ]:
uncertainty.beta